<a href="https://colab.research.google.com/github/itissandeep98/StackOverFlow-Tag-Predictor/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [ ]:
import numpy as np
import pandas as pd 
import csv,nltk,random,pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import pickle
import joblib
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def match_accuracy(y,y_pred):
    c= y==y_pred
    c=np.sum(c,axis=1)
    c[c >= 1] = 1
    return np.sum(c)/len(c)

# SVM


In [ ]:
def SVM(n,preprocessed_df,yx_multilabel,grid="0"):
    size=int(0.8*n)
    X_train=preprocessed_df.head(size)
    vectorizer = CountVectorizer(min_df=0.0009, max_features=13000,tokenizer = lambda x: x.split(), ngram_range=(1,4))
    vectorizer.fit(preprocessed_df['question'])
    X_test=preprocessed_df.tail(preprocessed_df.shape[0] - size)
    Y_train = yx_multilabel[0:size,:]
    Y_test = yx_multilabel[size:preprocessed_df.shape[0],:]
    x_train_multilabel = vectorizer.transform(X_train['question'])
    x_test_multilabel = vectorizer.transform(X_test['question'])

    if (grid=="0"):
        clf = OneVsRestClassifier(SGDClassifier(alpha=0.001,penalty='l2'))
    else:
        model_to_set = OneVsRestClassifier(SGDClassifier(penalty='l2'))
        parameters= {"estimator__alpha": [1/10**(-i) for i in range(1,10)]}
        clf=GridSearchCV(model_to_set, param_grid=parameters)

    clf.fit(x_train_multilabel, Y_train)
    pickle.dump(clf,open("svm_"+ str(n) ,'wb'))
    # !cp /content/svm_"$n" /content/drive/MyDrive/MlProject/
    y_pred = clf.predict(x_test_multilabel)
    print("Accuracy :",metrics.accuracy_score(Y_test,y_pred))
    print("Macro f1 score :",metrics.f1_score(Y_test, y_pred, average = 'macro'))
    print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred, average = 'micro'))
    print("hamming loss :",metrics.hamming_loss(Y_test,y_pred))
    # print("1 match accuracy :", match_accuracy(Y_test,y_pred))

In [ ]:
warnings.filterwarnings("ignore")

for i in range(10000,100001,10000):
    print("Dataset size is ",i)
    preprocessed_df=pd.read_csv("/content/drive/MyDrive/MlProject/data/vectorised_X_"+str(i)+"k.csv")
    yx_multilabel=joblib.load("/content/drive/MyDrive/MlProject/data/vectorised_y_"+str(i)+".pkl")
    SVM(i,preprocessed_df,yx_multilabel)   
    print("----------------------------------------------") 

Dataset size is  10000
Accuracy : 0.3807615230460922
Macro f1 score : 0.3986084438327051
Micro f1 scoore : 0.4431575739103931
hamming loss : 0.018306613226452904
----------------------------------------------
Dataset size is  20000
Accuracy : 0.4483798040693293
Macro f1 score : 0.40585598977368437
Micro f1 scoore : 0.48367313111221605
hamming loss : 0.015649334338106002
----------------------------------------------
Dataset size is  30000
Accuracy : 0.45915966386554624
Macro f1 score : 0.3986015293244122
Micro f1 scoore : 0.48934010152284263
hamming loss : 0.015216806722689076
----------------------------------------------
Dataset size is  40000
Accuracy : 0.46800962878499935
Macro f1 score : 0.38972557853098166
Micro f1 scoore : 0.48082569623632
hamming loss : 0.014785252755606233
----------------------------------------------
Dataset size is  50000
Accuracy : 0.48267452494665175
Macro f1 score : 0.3981323967266199
Micro f1 scoore : 0.49551090209491233
hamming loss : 0.014388781627883